In [6]:
!pip install duckdb==0.5.0
!pip install watchdog

In [3]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, PatternMatchingEventHandler
import duckdb

path = "landing/persistent/"
global con
con = duckdb.connect(database='formatted/my-db.duckdb', read_only=False)

class EventHandler(PatternMatchingEventHandler):
    def __init__(self):
        PatternMatchingEventHandler.__init__(
            self,
            patterns=["*.csv"],
            ignore_directories=True,
            case_sensitive=False,
        )
    
    
    def on_created(self, event):
        filename = event.src_path.split("/")[-1]
        filename_trunc = filename.split(".")[0]
        try:
            cursor = con.cursor()
            cursor.execute(f"CREATE TABLE {filename_trunc} AS SELECT * FROM read_csv_auto(?);", [ 
                                                                         event.src_path])
            cursor.execute("show tables")
            print(cursor.fetchall())
        except:
            pass
        finally:
            cursor.close()
        
        
event_handler = EventHandler()
observer = Observer()
observer.schedule(event_handler, path, recursive=True)
observer.start()

try:
    while True:
        time.sleep(1)
        
except KeyboardInterrupt:
    observer.stop()
observer.join()

[('chocolate_part_1',)]
[('chocolate_part_1',), ('chocolate_part_2',)]
[('chocolate_part_1',), ('chocolate_part_2',), ('freedom_part_1',)]
[('chocolate_part_1',), ('chocolate_part_2',), ('freedom_part_1',), ('freedom_part_2',)]
